In [1]:
#Load required packages
%matplotlib inline
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
import cftime
from tqdm import tqdm
import cmocean as cm
import sys, os
from datetime import datetime

from dask.distributed import Client

In [2]:
# Load workers:
client = Client(threads_per_worker=1)
client

/g/data/xp65/public/apps/med_conda/envs/analysis3-24.12/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40435 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/40435/status,
Dashboard: /proxy/40435/status,Workers: 28
Total threads: 28,Total memory: 251.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44065,Workers: 28
Dashboard: /proxy/40435/status,Total threads: 28
Started: Just now,Total memory: 251.19 GiB
Comm: tcp://127.0.0.1:34591,Total threads: 1
Dashboard: /proxy/42197/status,Memory: 8.97 GiB
Nanny: tcp://127.0.0.1:43363,


In [4]:
time = slice('1982-01-01','2023-12-29')
xt_coord = slice(-280,80)
yt_coord = slice(-81,-60)

clim_time = slice('1982-01-01','2011-12-31')

import cosima_cookbook as cc
db = 'local_cc_scratch_Ryan_data.db'
session = cc.database.create_session(db)


## SST

In [5]:
## CLIMATOLOGIES
pattern = '/g/data/e14/ts7341/Analysis_Using_Daily_JRA/JRA_SST_Processed/Climatology/*nc'

import xarray as xr 

def select_sst_only(ds):
    return ds[["sst"]]

Seas = xr.open_mfdataset(
          pattern,
          combine="by_coords",        
          parallel=True,
          preprocess=select_sst_only
).sel(lat = yt_coord, lon = xt_coord)
Seas

<xarray.Dataset> Size: 411MB
Dimensions:    (lat: 195, lon: 1440, dayofyear: 366)
Coordinates:
  * lat        (lat) float64 2kB -80.97 -80.87 -80.76 ... -60.33 -60.21 -60.08
  * lon        (lon) float64 12kB -279.9 -279.6 -279.4 ... 79.38 79.62 79.88
  * dayofyear  (dayofyear) int64 3kB 1 2 3 4 5 6 7 ... 361 362 363 364 365 366
Data variables:
    sst        (lat, lon, dayofyear) float32 411MB dask.array<chunksize=(195, 240, 366), meta=np.ndarray>

In [6]:
## THRESHOLDS
pattern = "/g/data/e14/ts7341/Analysis_Using_Daily_JRA/JRA_SST_Processed/Thresholds90thPercentile/*.nc"

Thresh = xr.open_mfdataset(
    pattern,
    combine="by_coords",        
    parallel=True,              
)

Thresh = Thresh.sel(lat = yt_coord).sel(lon = xt_coord)

In [7]:
# Shift 'Thresh' dayofyear from 0-365 to 1-366
Thresh = Thresh.assign_coords(dayofyear=Thresh['dayofyear'] + 1)
Thresh

<xarray.Dataset> Size: 822MB
Dimensions:                        (lon: 1440, lat: 195, dayofyear: 366)
Coordinates:
  * lon                            (lon) float64 12kB -279.9 -279.6 ... 79.88
  * lat                            (lat) float64 2kB -80.97 -80.87 ... -60.08
  * dayofyear                      (dayofyear) int64 3kB 1 2 3 4 ... 364 365 366
Data variables:
    __xarray_dataarray_variable__  (lat, lon, dayofyear) float64 822MB dask.array<chunksize=(195, 240, 366), meta=np.ndarray>

In [ ]:
ds = cc.querying.getvar(expt='025deg_jra55_iaf_cycle6_online_mlt', variable='sst', 
                          session=session, frequency='1 daily',
                          attrs={'cell_methods': 'time: mean'},
                          chunks={'time': -1, 'xt_ocean': '400MB', 'yt_ocean': '400MB'}).\
                          sel(time = time).\
                          sel(yt_ocean = yt_coord).sel(xt_ocean = xt_coord).\
                          load()

In [ ]:
print(ds)

In [ ]:
# # Rename dimensions
ds = ds.rename({'xt_ocean': 'lon', 'yt_ocean': 'lat'})

# # Convert surface_temp from Kelvin to Celsius
ds_input = ds - 273.15
ds_input.attrs['units'] = 'deg C'  # Update units attribute

# # Display the modified dataset structure
ds_input = ds_input

In [ ]:
ds_input = ds_input.assign_coords(dayofyear=ds_input['time'].dt.dayofyear)
print(ds_input)

In [ ]:
import xarray as xr

Seas_aligned = Seas.interp(
    dayofyear=xr.DataArray(ds_input['dayofyear'].values, dims="time")
)
Thresh_aligned = Thresh.sel(dayofyear=ds_input['dayofyear']).chunk({'time':-1})

In [ ]:
Thresh_aligned = Thresh_aligned.__xarray_dataarray_variable__
Thresh_aligned

In [ ]:
Seas_aligned = Seas_aligned.sst

In [ ]:
Seas_aligned = Seas.sel(dayofyear=ds_input['dayofyear'])
Seas_aligned = Seas_aligned.sst
Seas_aligned

In [ ]:
SSTa = ds_input - Seas_aligned
print(SSTa) ## SSTA loaded

In [ ]:
SSTa = SSTa.chunk({'time':-1,'lon':'300MB','lat':'300MB'})

In [ ]:
SSTa

In [ ]:
Severity_ds =  xr.Dataset({
    'thresh_aligned': Thresh_aligned, 
    'sst': ds_input,
    'ssta': SSTa,
#    'severity': ts_severity
}).chunk({'time': -1})

In [19]:
Severity_ds

<xarray.Dataset> Size: 5GB
Dimensions:         (lon: 320, lat: 66, time: 15338)
Coordinates:
  * lon             (lon) float64 3kB -209.9 -209.6 -209.4 ... -130.4 -130.1
  * lat             (lat) float64 528B -76.96 -76.85 -76.75 ... -70.2 -70.09
    dayofyear       (time) int64 123kB dask.array<chunksize=(15338,), meta=np.ndarray>
  * time            (time) datetime64[ns] 123kB 1982-01-01T12:00:00 ... 2023-...
Data variables:
    thresh_aligned  (lat, lon, time) float64 3GB dask.array<chunksize=(66, 200, 15338), meta=np.ndarray>
    sst             (time, lat, lon) float32 1GB dask.array<chunksize=(15338, 66, 320), meta=np.ndarray>
    ssta            (time, lat, lon) float32 1GB dask.array<chunksize=(15338, 44, 110), meta=np.ndarray>

In [ ]:
Severity_ds = Severity_ds.sel(time = slice('2016','2024'))

## NOAA

In [3]:
import glob
import xarray as xr
import re

# Get a list of all NetCDF files
filelist = glob.glob('/g/data/e14/asg561/MHW_xarray/mhw_data_90pc/mhw*.nc')

# Define a regex pattern to match latitudes -90to-70 or -70to-50
pattern = re.compile(r'\.(-90to-70|-70to-50)\.')

# Filter the file list based on the pattern
filtered_files = [f for f in filelist if pattern.search(f)]

# Check if there are matching files
if filtered_files:
    # Open only the filtered files
    satellite_data = xr.open_mfdataset(filtered_files, combine='by_coords')
    print(satellite_data)
else:
    print("No matching files found!")

satellite_data

<xarray.Dataset> Size: 59GB
Dimensions:         (time: 15688, lat: 160, lon: 1440, time366: 366)
Coordinates:
  * lat             (lat) float32 640B -89.88 -89.62 -89.38 ... -50.38 -50.12
  * lon             (lon) float32 6kB 0.125 0.375 0.625 ... 359.4 359.6 359.9
  * time            (time) datetime64[ns] 126kB 1981-12-30 ... 2024-12-11
  * time366         (time366) datetime64[ns] 3kB 1983-12-30 ... 1984-12-29
Data variables:
    severity        (time, lat, lon) float32 14GB dask.array<chunksize=(1, 80, 120), meta=np.ndarray>
    severity_index  (time, lat, lon) float32 14GB dask.array<chunksize=(1, 80, 120), meta=np.ndarray>
    ssta            (time, lat, lon) float32 14GB dask.array<chunksize=(1, 80, 120), meta=np.ndarray>
    sst             (time, lat, lon) float32 14GB dask.array<chunksize=(1, 80, 120), meta=np.ndarray>
    climatology90   (time366, lat, lon) float32 337MB dask.array<chunksize=(1, 80, 120), meta=np.ndarray>
    climatology     (time366, lat, lon) float32 337MB d

<xarray.Dataset> Size: 59GB
Dimensions:         (time: 15688, lat: 160, lon: 1440, time366: 366)
Coordinates:
  * lat             (lat) float32 640B -89.88 -89.62 -89.38 ... -50.38 -50.12
  * lon             (lon) float32 6kB 0.125 0.375 0.625 ... 359.4 359.6 359.9
  * time            (time) datetime64[ns] 126kB 1981-12-30 ... 2024-12-11
  * time366         (time366) datetime64[ns] 3kB 1983-12-30 ... 1984-12-29
Data variables:
    severity        (time, lat, lon) float32 14GB dask.array<chunksize=(1, 80, 120), meta=np.ndarray>
    severity_index  (time, lat, lon) float32 14GB dask.array<chunksize=(1, 80, 120), meta=np.ndarray>
    ssta            (time, lat, lon) float32 14GB dask.array<chunksize=(1, 80, 120), meta=np.ndarray>
    sst             (time, lat, lon) float32 14GB dask.array<chunksize=(1, 80, 120), meta=np.ndarray>
    climatology90   (time366, lat, lon) float32 337MB dask.array<chunksize=(1, 80, 120), meta=np.ndarray>
    climatology     (time366, lat, lon) float32 337MB dask.array<chunksize=(1, 80, 120), meta=np.ndarray>
Attributes:
    history:  Created in regional_MHW_hdd4_anomaly.py pctile=902025-01-06 14:...

In [4]:
ssta = satellite_data.ssta

In [20]:
ssta = ssta.sel(lon = slice(360-210,360-130)).sel(lat = slice(-77,-71))

# --- Climatology 90 ---
clim90 = satellite_data.climatology90.sel(lon = slice(360-210,360-130)).sel(lat = slice(-77,-71))

# --- Climatology ---
clim = satellite_data.climatology.sel(lon = slice(360-210,360-130)).sel(lat = slice(-77,-71))

In [24]:
import xarray as xr

# --- 1. Assign DOY (day-of-year) to `temp` ---
ssta = ssta.assign_coords(doy=ssta['time'].dt.dayofyear)

# --- 2. Assign DOY 1–366 to the climatology datasets ---

# This assumes your climatology and climatology90 each have 366 entries
doy_full = np.arange(1, 367)

# Assign DOY as a coordinate
climatology90 = clim90.assign_coords(doy=('time366', doy_full))
climatology = clim.assign_coords(doy=('time366', doy_full))
 
# Swap dimension so we can index by DOY instead of time366
climatology90 = climatology90.swap_dims({'time366': 'doy'})
climatology = climatology.swap_dims({'time366': 'doy'})

# --- 3. Align climatology with temp using DOY ---

# Select climatology using the DOY from temp, aligned over time
sst_clim = climatology.sel(doy=xr.DataArray(ssta['doy'].values, dims='time'))
sst_clim = sst_clim.assign_coords(time=ssta['time'])

# Repeat for threshold climatology
sst_clim90 = climatology.sel(doy=xr.DataArray(ssta['doy'].values, dims='time'))
sst_clim90 = sst_clim90.assign_coords(time=ssta['time'])

sst_clim90.load()
sst_clim.load()
ssta.load()

<xarray.DataArray 'ssta' (time: 15688, lat: 24, lon: 320)> Size: 482MB
array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.02441406,
          0.02246094,  0.02929688],
        [ 0.        ,  0.        ,  0.        , ..., -0.00292969,
          0.00488281,  0.00292969],
        [ 0.        ,  0.        ,  0.        , ..., -0.03222656,
         -0.03417969, -0.03710938]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
...
        [ 0.        ,  0.        ,  0.        , ..., -0.15820312,
         -0.15722656, -0.15625   ],
        [ 0.        ,  0.        ,  0.        , ..., -0.16601562,
         -0.1640625 , -0.16113281],
        [ 0.        ,  0.        ,  0.        , ..., -0.17675781,
         -0.17382812, -0.16992188]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ..., -0.16113281,
         -0.16015625, -0.15820312],
        [ 0.        ,  0.        ,  0.        , ..., -0.16894531,
         -0.16699219, -0.1640625 ],
        [ 0.        ,  0.        ,  0.        , ..., -0.18066406,
         -0.17773438, -0.17382812]]], dtype=float32)
Coordinates:
  * lat      (lat) float32 96B -76.88 -76.62 -76.38 ... -71.62 -71.38 -71.12
  * lon      (lon) float32 1kB 150.1 150.4 150.6 150.9 ... 229.4 229.6 229.9
  * time     (time) datetime64[ns] 126kB 1981-12-30 1981-12-31 ... 2024-12-11
    doy      (time) int64 126kB 364 365 1 2 3 4 5 ... 341 342 343 344 345 346
Attributes:
    units:      degrees C
    long_name:  Daily sea surface temperature anomaly

In [26]:
Severity_ds =  xr.Dataset({
    'thresh_aligned': sst_clim90, 
    'sst': satellite_data.sst.sel(lon = slice(360-210 ,360-130)).sel(lat = slice(-77,-71)),
    'ssta': ssta,
#    'severity': ts_severity
}).chunk({'time': -1})

In [29]:
Severity_ds.se

<xarray.Dataset> Size: 1GB
Dimensions:         (lat: 24, lon: 320, time: 15688)
Coordinates:
  * lat             (lat) float32 96B -76.88 -76.62 -76.38 ... -71.38 -71.12
  * lon             (lon) float32 1kB 150.1 150.4 150.6 ... 229.4 229.6 229.9
    time366         (time) datetime64[ns] 126kB dask.array<chunksize=(15688,), meta=np.ndarray>
    doy             (time) int64 126kB dask.array<chunksize=(15688,), meta=np.ndarray>
  * time            (time) datetime64[ns] 126kB 1981-12-30 ... 2024-12-11
Data variables:
    thresh_aligned  (time, lat, lon) float32 482MB dask.array<chunksize=(15688, 24, 320), meta=np.ndarray>
    sst             (time, lat, lon) float32 482MB dask.array<chunksize=(15688, 24, 120), meta=np.ndarray>
    ssta            (time, lat, lon) float32 482MB dask.array<chunksize=(15688, 24, 320), meta=np.ndarray>

## Metrics

In [32]:
import xarray as xr
import numpy as np
import datetime as dt
import bottleneck as bn
import marineHeatWaves as mhw
import pandas as pd
from pathlib import Path

def detectMHWs_fromThreshold(time, sst, threshold, l_return=2000, minDuration=5, maxGap=2, minExceed = 0.1):
    """
    Detect MHWs based on SST exceeding a threshold time series.
    """
    t = time
    boolSever = sst > (threshold + minExceed)

    # Preallocate dictionary
    mhw = {}
    mhw['index_start'] = np.empty(l_return) * np.nan
    mhw['index_end'] = np.empty(l_return) * np.nan
    mhw['date_start'] = np.empty(l_return) * np.nan
    mhw['date_end'] = np.empty(l_return) * np.nan
    mhw['duration'] = np.empty(l_return) * np.nan
    ia = np.asarray(boolSever)
    n = len(ia)

    if not np.isnan(ia).all():
        y = ia[1:] != ia[:-1]
        i = np.append(np.where(y), n-1)
        l_cont = np.diff(np.append(-1, i))
        p = np.append(0, i+1)[:-1]
        type_vals = ia[i]

        b_mhw = np.where((l_cont >= minDuration) & (type_vals == 1))[0]
        i_start = p[b_mhw]
        i_end = i[b_mhw]

        l_gap = i_start[1:] - i_end[:-1] - 1
        boolGap = l_gap <= maxGap
        if any(boolGap):
            indGap = np.where(boolGap)[0]
            i_start = np.delete(i_start, indGap+1)
            i_end = np.delete(i_end, indGap)

        n_events = len(i_start)
        mhw['index_start'][:n_events] = i_start
        mhw['index_end'][:n_events] = i_end
        mhw['date_start'][:n_events] = t[i_start]
        mhw['date_end'][:n_events] = t[i_end]
        mhw['duration'][:n_events] = i_end - i_start + 1

    ds_mhw = xr.Dataset({var: ('event', data) for var, data in mhw.items()},
                        coords={'event': ('event', range(l_return))})
    return ds_mhw.to_array()

def add_metrics_MHWs(time, ssta, date_start, date_end, l_return=2000):
    """
    Calculate more metrics, such as peak date and intensities.  Could add more in this function.
    This function is also called in an apply_ufunc call, to parallelize things.
    Inputs:
        - time: a np.datetime64 vector
        - ssta: sea surface temperature anomaly (vector)
        - date_start: np.datetime64 vector containing the start date of all detected MHW events (for this one ssta)
        - date_end: np.datetime64 vector containing the end date of all detected MHW events (for this one ssta)
        - l_return: the maximum number of expected MHW events (to pre-allocate the output).
    Output:
        - Numpy array containing more metrics, including peak date and various intensities.
    """
    # Force numpy arrays
    time_np = time                # time
    ssta_np = np.asarray(ssta)                # ssta
    date_start_np = np.asarray(date_start)    # date_start
    date_end_np = np.asarray(date_end)        # date_end
    # Make sure it is only 2D, i.e. only one grid cell.
    if len(ssta_np.shape)>2:
        print('Dataset has too many dimensions, computation will be too heavy and crash.')
        print('Try to vectorize or loop over grid cells.')
        return
    # Start a new dictionary
    mhw_out = {}
    mhw_out['index_peak'] = np.empty(l_return) * np.nan
    mhw_out['date_peak'] = np.empty(l_return) * np.nan
    mhw_out['intensity_max'] = np.empty(l_return) * np.nan
    mhw_out['intensity_mean'] = np.empty(l_return) * np.nan
    mhw_out['intensity_cumul'] = np.empty(l_return) * np.nan
    if ~np.isnan(ssta_np).all(): # Make sure it is not a nan array
        boolnan = ~np.isnan(date_start_np)
        dstart = date_start_np[boolnan]
        dend = date_end_np[boolnan]
        # For each event, create a time series of ssta with all values out of MHWs naned-out
        np_ssta_evts = np.where((time_np[:, np.newaxis]>=dstart) & (time_np[:, np.newaxis]<=dend),
                                ssta_np[:, np.newaxis],np.nan)
        # Use the argmax to extract the ssta maximum during the mhw, then convert to a date
        n_events = bn.nansum(boolnan)
        index_peak = bn.nanargmax(np_ssta_evts,axis=0)
        mhw_out['index_peak'][:n_events] = index_peak
        date_peak = time_np[index_peak]
        mhw_out['date_peak'][:n_events] = date_peak
        # # Now calculate intensities
        mhw_out['intensity_max'][:n_events] = bn.nanmax(np_ssta_evts,axis=0)
        mhw_out['intensity_mean'][:n_events] = bn.nanmean(np_ssta_evts,axis=0)
        mhw_out['intensity_cumul'][:n_events] = bn.nansum(np_ssta_evts,axis=0)
        # Finally convert to a ds to send back.
    ds_mhw_out = xr.Dataset({var: ('event', data) for var, data in mhw_out.items()},
                            coords={'event':('event',range(l_return))})
    return ds_mhw_out.to_array()


def calculate_MHWs_metrics(ds, maxEvt=2000, minExceed=0.1):
    """
    Vectorized MHW detection across grid cells using SST and thresh_aligned
    """
    mhw_da = xr.apply_ufunc(
        detectMHWs_fromThreshold,
        ds.time, ds.sst, ds.thresh_aligned,
        kwargs={'l_return': maxEvt, 'minExceed': minExceed},
        input_core_dims=[['time'], ['time'], ['time']],
        output_core_dims=[['variable', 'event']],
        dask="parallelized",
        dask_gufunc_kwargs={"output_sizes": {"event": maxEvt, "variable": 5}},
        vectorize=True,
        output_dtypes=[float],  # <- specify float for numeric outputs
    )


    # Convert DataArray to Dataset
    ds_mhw = mhw_da.assign_coords(
        {'variable': ('variable', ['index_start', 'index_end', 'date_start', 'date_end', 'duration']),
         'event': ('event', range(maxEvt))}
    ).to_dataset(dim='variable')

    ds_mhw = ds_mhw.dropna('event', how='all')
    ds_mhw['date_start'] = ds_mhw.date_start.astype('datetime64[ns]')
    ds_mhw['date_end'] = ds_mhw.date_end.astype('datetime64[ns]')
    ds_mhw['n_event'] = ds_mhw.index_start.notnull().sum(dim='event')

    ds_mhw_more = xr.apply_ufunc(add_metrics_MHWs, ds.time, ds.ssta, 
                                 ds_mhw.date_start, ds_mhw.date_end, 
                                 kwargs={'l_return':len(ds_mhw.event)},
                                 input_core_dims=[['time'],['time'],['event'],['event']],
                                 output_core_dims=[['variable','event']],
                                 dask="parallelized",
                                 dask_gufunc_kwargs={"output_sizes": {"variable": 5, "event": len(ds_mhw.event)}},
                                 output_dtypes=['float'],
                                 vectorize=True)
    # Convert from a dataArray to a Dataset
    ds_mhw_more = ds_mhw_more.assign_coords({'variable':('variable',
                                                         ['index_peak','date_peak','intensity_max',
                                                          'intensity_mean','intensity_cumul']),
                                             'event':('event',range(len(ds_mhw.event)))}).to_dataset(dim='variable')
    # Merge both datasets
    ds_mhw_more['date_peak'] = ds_mhw_more.date_peak.astype('datetime64[ns]')
    ds_mhw_full = ds_mhw.merge(ds_mhw_more)
    return ds_mhw_full


In [33]:
mhws2 = calculate_MHWs_metrics(Severity_ds)

/g/data/xp65/public/apps/med_conda/envs/analysis3-24.12/lib/python3.11/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 467.59 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/g/data/xp65/public/apps/med_conda/envs/analysis3-24.12/lib/python3.11/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 467.59 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/g/data/xp65/public/apps/med_conda/envs/analysis3-24.12/lib/python3.11/site-packages/distributed/c

In [34]:
mhws2.load()

/g/data/xp65/public/apps/med_conda/envs/analysis3-24.12/lib/python3.11/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 0.91 GiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


<xarray.Dataset> Size: 141MB
Dimensions:          (lat: 24, lon: 320, event: 229)
Coordinates:
  * lat              (lat) float32 96B -76.88 -76.62 -76.38 ... -71.38 -71.12
  * lon              (lon) float32 1kB 150.1 150.4 150.6 ... 229.4 229.6 229.9
  * event            (event) int64 2kB 0 1 2 3 4 5 6 ... 223 224 225 226 227 228
Data variables:
    index_start      (lat, lon, event) float64 14MB nan nan nan ... nan nan nan
    index_end        (lat, lon, event) float64 14MB nan nan nan ... nan nan nan
    date_start       (lat, lon, event) datetime64[ns] 14MB NaT NaT ... NaT NaT
    date_end         (lat, lon, event) datetime64[ns] 14MB NaT NaT ... NaT NaT
    duration         (lat, lon, event) float64 14MB nan nan nan ... nan nan nan
    n_event          (lat, lon) int64 61kB 0 0 0 0 0 0 0 ... 89 85 78 73 74 77
    index_peak       (lat, lon, event) float64 14MB nan nan nan ... nan nan nan
    date_peak        (lat, lon, event) datetime64[ns] 14MB NaT NaT ... NaT NaT
    intensity_max    (lat, lon, event) float64 14MB nan nan nan ... nan nan nan
    intensity_mean   (lat, lon, event) float64 14MB nan nan nan ... nan nan nan
    intensity_cumul  (lat, lon, event) float64 14MB nan nan nan ... nan nan nan

In [26]:
cell_lat = -66.5  # example latitude
cell_lon = 10.0   # example longitude

onset_times = mhws2['date_start'].sel(lat=cell_lat, lon=cell_lon, method = 'nearest')
print(onset_times.values)  # array of datetime64[ns] for all events at that cell

onset_times = mhws2['duration'].sel(lat=cell_lat, lon=cell_lon, method = 'nearest')
print(onset_times.values)  # array of datetime64[ns] for all events at that cell


['1985-10-31T12:00:00.000000000' '1991-12-29T12:00:00.000000000'
 '1993-01-06T12:00:00.000000000' '1993-02-23T12:00:00.000000000'
 '1993-11-18T12:00:00.000000000' '1993-12-21T12:00:00.000000000'
 '2000-12-09T12:00:00.000000000' '2001-02-26T12:00:00.000000000'
 '2002-01-26T12:00:00.000000000' '2002-02-18T12:00:00.000000000'
 '2002-03-04T12:00:00.000000000' '2002-12-31T12:00:00.000000000'
 '2003-01-15T12:00:00.000000000' '2003-03-26T12:00:00.000000000'
 '2004-12-06T12:00:00.000000000' '2005-11-17T12:00:00.000000000'
 '2006-01-31T12:00:00.000000000' '2016-10-06T12:00:00.000000000'
 '2022-02-24T12:00:00.000000000' '2023-03-13T12:00:00.000000000'
 '2023-11-28T12:00:00.000000000' '2023-12-12T12:00:00.000000000'
                           'NaT'                           'NaT'
                           'NaT'                           'NaT'
                           'NaT'                           'NaT'
                           'NaT'                           'NaT'
                         

In [35]:
mhws2.to_netcdf('/scratch/e14/ts7341/MHW_Statistics_Mask/full_MHW_data.nc')

In [49]:
import xarray as xr
ds = xr.open_dataset('/scratch/e14/ts7341/MHW_Statistics_Mask/full_MHW_data_1982_2024_RS_NOAA.nc')

In [50]:
ds

<xarray.Dataset> Size: 141MB
Dimensions:          (lat: 24, lon: 320, event: 229)
Coordinates:
  * lat              (lat) float32 96B -76.88 -76.62 -76.38 ... -71.38 -71.12
  * lon              (lon) float32 1kB 150.1 150.4 150.6 ... 229.4 229.6 229.9
  * event            (event) int64 2kB 0 1 2 3 4 5 6 ... 223 224 225 226 227 228
Data variables:
    index_start      (lat, lon, event) float64 14MB ...
    index_end        (lat, lon, event) float64 14MB ...
    date_start       (lat, lon, event) datetime64[ns] 14MB ...
    date_end         (lat, lon, event) datetime64[ns] 14MB ...
    duration         (lat, lon, event) float64 14MB ...
    n_event          (lat, lon) int64 61kB ...
    index_peak       (lat, lon, event) float64 14MB ...
    date_peak        (lat, lon, event) datetime64[ns] 14MB ...
    intensity_max    (lat, lon, event) float64 14MB ...
    intensity_mean   (lat, lon, event) float64 14MB ...
    intensity_cumul  (lat, lon, event) float64 14MB ...